### Activity 5.01: Reading Tabular Data from a Web Page and Creating DataFrames



In [49]:
# Ignore all Warnings
import warnings
warnings.filterwarnings('ignore')

In [50]:
# Make a get requests to Wikipedia using Requests

import requests
page= requests.get ('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)')
print (f'Requests status Code:',page.status_code)


Requests status Code: 200


In [51]:
# Read the page using bs4

from bs4 import BeautifulSoup
soup = BeautifulSoup(page.text,'html.parser') #Read the page from requests response using html parser

In [52]:
# Count the no of tablular data 

table = soup.findAll("table")  #Scan all the table
print (f'Total Tables:',len(table)) 

Total Tables: 7


In [53]:
# Find the right table by finding no of rows and columns for each table

cntr = 0 
num_cols = 0

for data in table:
    cntr += 1  #Increment the Counter to display Table index
    rows = data.find_all('tr') #Find all the tr to find no of rows
    
    for row in rows: #To find no of columns loop through each row
        cols = row.find_all(['th']) #Find all the header
        num_cols = max(num_cols, len(cols)) # Get the max of the count

    print (f'Table {cntr}: \n No of rows: {len(rows)} \n No of columns: {num_cols}')

Table 1: 
 No of rows: 2 
 No of columns: 0
Table 2: 
 No of rows: 1 
 No of columns: 0
Table 3: 
 No of rows: 212 
 No of columns: 6
Table 4: 
 No of rows: 10 
 No of columns: 6
Table 5: 
 No of rows: 9 
 No of columns: 6
Table 6: 
 No of rows: 14 
 No of columns: 6
Table 7: 
 No of rows: 2 
 No of columns: 6


From the above we can identity that we need to use Table 3

In [54]:
import pandas as pd
import re

# Define the regular expression pattern for removing unwanted chars in year
pattern = r'\[n\s+\d+\]'

rows = table[2].find_all('tr')[2:]  # Get all the rows other than header for the table 3

# Extract data
data = []
for row in rows:
    cols = [re.sub(pattern, '',col.get_text(strip=True)).replace(',', '') for col in row.find_all('td')]  #Extract human readable text from td tag and replace value like [n 1], ","
    data.append(cols) #Append all the data
    
# Create main dataframe
df = pd.DataFrame(data, columns=[
    'Country', 'IMF_Forecast', 'IMF_Year',
    'WB_Estimate', 'WB_Year', 'UN_Estimate', 'UN_Year'
])

# Convert numeric columns
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Create separate dataframes
df_imf = df[['Country', 'IMF_Forecast', 'IMF_Year']]
df_wb = df[['Country', 'WB_Estimate', 'WB_Year']]
df_un = df[['Country', 'UN_Estimate', 'UN_Year']]

#Print different dataframe
print ("IMF Forecast \n")
print(df_imf.head (10))
print("\n")

print ("World Bank Forecast \n")
print(df_wb.head (10))
print("\n")

print ("United Nation Forecast \n")
print(df_un.head (10))
print("\n")



IMF Forecast 

          Country  IMF_Forecast  IMF_Year
0           World   115494312.0    2025.0
1   United States    30338000.0    2025.0
2           China    19535000.0    2025.0
3         Germany     4922000.0    2025.0
4           Japan     4390000.0    2025.0
5           India     4270000.0    2025.0
6  United Kingdom     3731000.0    2025.0
7          France     3284000.0    2025.0
8           Italy     2460000.0    2025.0
9          Canada     2331000.0    2025.0


World Bank Forecast 

          Country  WB_Estimate  WB_Year
0           World  105435540.0   2023.0
1   United States   27360935.0   2023.0
2           China   17794782.0   2023.0
3         Germany    4456081.0   2023.0
4           Japan    4212945.0   2023.0
5           India    3549919.0   2023.0
6  United Kingdom    3340032.0   2023.0
7          France    3030904.0   2023.0
8           Italy    2254851.0   2023.0
9          Canada    2140086.0   2023.0


United Nation Forecast 

          Country  UN_Estimate  